# Prompt Engineering, explored with Semantic Kernel and Azure OpenAI

To quickly get started, follow these steps:

1. Install [Polyglot notebooks extension](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) in VSCode.
2. [Create a new Azure OpenAI service (or use an existing OpenAI service)](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart?tabs=command-line&pivots=programming-language-studio#prerequisites).
3. [Deploy the `gpt-35-turbo` and `text-embeddings-ada-002` models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#working-with-models).
4. [Create an Azure Cognitive Search instance and enable the Semantic Search capability](https://learn.microsoft.com/en-us/azure/search/semantic-search-overview#enable-semantic-search).
5. Copy the `.env.example` file from the parent folder to `dotnet/.env` and paste the corresponding values from the resources you provisioned in the earlier steps.
6. Click on `Run All`.


> You will need an [.Net 7 SDK](https://dotnet.microsoft.com/en-us/download) and [Polyglot](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) to get started with this notebook using .Net Interactive

# Background

## What is Prompt Engineering?
Prompt engineering is an iterative approach for crafting and refining prompts to enhance interactions with Large Language Models (LLMs). Mastery of prompt engineering is key to unlocking the full potential of LLMs in various applications. This has been pivotal in achieving advanced use cases in Microsoft Copilots.

This notebook serves as your go-to resource for effective prompt engineering techniques.

### Best Practices: Insights from Azure OpenAI

#### Be Specific and Descriptive
Craft your prompts to be both specific and descriptive to minimize ambiguity. Using analogies or metaphors can aid in making the prompts more understandable and relatable to the model.

#### Be Repetitive
- **Repeat**: Reiterate key instructions to ensure clarity and focus in the model's output.
- **Order Matters**: The sequence in which you present instructions can influence the model’s response due to its recency bias.

#### Space Efficiency
- **Token Limitations**: Be aware of the token limits for the model you are invoking.
- **Data Formats**: Opt for tabular formats over JSON for greater space efficiency.
- **White Space**: Use space judiciously, as each extra space counts as a token and can limit the model's performance.



### High Level Objectives
1. Include
	> Include instructions of requesting the model not to make up stuff but stay with facts.
1. Restrict
	> Restrict the output (e.g., choose from a confined list instead of generating free form strings)
1. Add
	> Add Chain of Thought style of instruction, "Solve the problem step by step."
1. Repeat
	> Repeat most important instructions in the prompt a couple of times.
1. Position
	> Position most important instructions in the last making use of latency effect.


## Let's Get Started

We will be utilizing **Semantic Kernel** to orchestrate interactions with the `gpt-35-turbo` model, which is deployed on **Azure OpenAI** for brevity. Alternatively, you can also use the **Azure OpenAI SDK** for model orchestration.


## Setup the Kernel to Interact with Azure OpenAI

Install dependencies and import libraries.

In [1]:
#r "nuget: dotenv.net"
dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Installed Packages dotenv.net, 3.1.3

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta6"
#r "nuget: Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta6"

Installed Packages Microsoft.SemanticKernel, 1.0.0-beta6 Microsoft.SemanticKernel.Connectors.Memory.AzureCognitiveSearch, 1.0.0-beta6

In [3]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;

### Setup
Initialize the kernel using Azure OpenAI Chat completion model and memory store for RaG related prompting

In [4]:
var builder = new KernelBuilder();

builder
.WithAzureOpenAIChatCompletionService(
    env["AZURE_OPENAI_CHAT_MODEL"],
    env["AZURE_OPENAI_ENDPOINT"],
    env["AZURE_OPENAI_API_KEY"]
);

IKernel kernel = builder.Build();

In [6]:
using Microsoft.SemanticKernel.Plugins.Memory;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
var memoryBuilder = new MemoryBuilder();
memoryBuilder
    .WithAzureOpenAITextEmbeddingGenerationService(
        env["AZURE_OPENAI_EMBEDDING_MODEL"],
        env["AZURE_OPENAI_ENDPOINT"],
        env["AZURE_OPENAI_API_KEY"]
    )

    .WithMemoryStore(new VolatileMemoryStore());

var memory = memoryBuilder.Build();


## Start with clear instructions
> Tell the model the task you want it to do at the beginning of the prompt and repeat at the end 

In [9]:
string skPrompt = """
Your task is to verify if the statement "Several sources mention a chance of another large eruption" is supported by a specific quote from the following set of snippets.
{{$input}}
Is the statement "Several sources mention a chance of another large eruption" directly implied or stated by the snippets?
""";

var summaryFunction = kernel.CreateSemanticFunction(skPrompt, requestSettings: new OpenAIRequestSettings { MaxTokens = 2000, Temperature = 0.0, TopP = 0.5 });
string input = 
        """
        ---
        SNIPPETS
        [1] 14 percent chance of megaquake hitting Seattle, experts say
        SEATTLE - There's a 14 percent chance of a magnitude 9 Cascadia earthquake hitting Seattle in the next 50 years, the U.S. Geological Survey estimates. "Unfortunately, we are unable to...
        [2] Earthquake experts lay out latest outlook for Seattle's 'Really Big One’
        “We say that there's approximately a 14% chance of another approximately magnitude-9 earthquake occurring in the next 50 years,” said Erin Wirth, a geophysicist at the University of Washington...
        ---
        """;
var summaryResult = await kernel.RunAsync(input, summaryFunction);
var summary = summaryResult.GetValue<string>();

Console.WriteLine(summary);

No, the statement "Several sources mention a chance of another large eruption" is not directly implied or stated by the snippets. The snippets provided discuss the probability of a large earthquake, not an eruption. An eruption typically refers to a volcanic event, which is different from an earthquake.
